<a href="https://colab.research.google.com/github/ayush12gupta/CNN_models/blob/master/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np


In [0]:
class block(nn.Module):
    def __init__(self, inp, out, strides=1):
      super().__init__()

      self.cov1 = nn.Conv2d(inp, out, 3, padding=1, bias=False, stride=strides)
      self.bn1 = nn.BatchNorm2d(out)
      self.cov2 = nn.Conv2d(out, out, 3, padding=1, bias=False) # For same size of layer as the previour one
      self.bn2 = nn.BatchNorm2d(out)

      self.shortcut = nn.Sequential()
      if strides!=1 or inp!=out:
        self.shortcut = nn.Sequential(
            nn.Conv2d(inp, out,1,stride = strides,bias=False),
            nn.BatchNorm2d(out)
        )

    def forward(self,x):
        iden = x
        x = F.relu(self.bn1(self.cov1(x)))
        x = self.bn2(self.cov2(x))
        x += self.shortcut(iden)
        
        return F.relu(x)

In [0]:
class ResNet(nn.Module):
    def __init__(self, block, size, n,classnum=10):

        super().__init__()
        self.cov1 = nn.Conv2d(3, size[0], 3, bias=False)
        self.bn1 = nn.BatchNorm2d(size[0])
        self.cb1 = self.covBlock(block, size[0],n,stride=1)
        self.cb2 = self.covBlock(block, (size[0],size[1]),n,stride=2)
        self.cb3 = self.covBlock(block, (size[1],size[2]),n,stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(2)
        self.fc = nn.Linear(2*2*size[2],classnum)

    def covBlock(self,block,size,n,stride):

        if stride!=1:
          inp,out = size
        else:
          inp,out = size,size

        layer = []
        layer.append(block(inp,out,stride))
        for i in range(n-1):
          layer.append(block(out,out))

        cb = nn.Sequential(*layer)
        return cb  

    def forward(self, x):

        x = F.relu(self.bn1(self.cov1(x))) 
        x = self.cb1(x)
        x = self.cb2(x)
        x = self.cb3(x)  
        x = self.avgpool(x)
        res = 1
        for dim in x[0].shape:
            res *= dim  ##############
        x = x.view(-1, res)
        x = self.fc(x)
        x = F.softmax(x,dim=1)

        return x            

In [0]:
batch_size = 128

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((.5, .5, .5), (.5, .5, .5))])

trainset = torchvision.datasets.CIFAR10(root = './data', download = True, train = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root = './data', download = True, train = False, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = True, num_workers=2) # num workers is used to pre process data

Files already downloaded and verified
Files already downloaded and verified


In [0]:
model = ResNet(block, [16,32,64], 5).cuda()
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30,50,80], gamma=0.3)
device = 'cuda'

In [0]:
model.train()
print('Training Started')
for epoch in range(100):

    running_loss = 0.0
    correct, total = 0, 0
    correct_epoch, total_epoch = 0, 0
    scheduler.step()
    for i,data in enumerate(trainloader):

        inp,lab = data[0].to(device),data[1].to(device)

        optimizer.zero_grad()

        out = model(inp)
        loss = criterion(out,lab)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        c = (torch.argmax(out,1)==lab)
        correct+=torch.sum(c)
        total += 128
        correct_epoch += torch.sum(c)
        total_epoch += 128
        
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.7f accuracy: [%d, %d] (%d %%)' % (epoch + 1, i + 1, running_loss / 200, correct, total, 100*correct/total))
            running_loss = 0.0
            correct = 0
            total = 0
            #print('Epoch: {}   loss: {}  accuracy: {}'.format(epoch,loss,(correct/total)))

Training Started


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1,   200] loss: 2.2302093 accuracy: [5539, 25600] (21 %)
[2,   200] loss: 2.1473030 accuracy: [7865, 25600] (30 %)
[3,   200] loss: 2.0869989 accuracy: [9421, 25600] (36 %)
[4,   200] loss: 2.0507557 accuracy: [10368, 25600] (40 %)
[5,   200] loss: 2.0049857 accuracy: [11570, 25600] (45 %)
[6,   200] loss: 1.9676739 accuracy: [12551, 25600] (49 %)
[7,   200] loss: 1.9281547 accuracy: [13546, 25600] (52 %)
[8,   200] loss: 1.8856165 accuracy: [14657, 25600] (57 %)
[9,   200] loss: 1.8576131 accuracy: [15363, 25600] (60 %)
[10,   200] loss: 1.8265894 accuracy: [16185, 25600] (63 %)
[11,   200] loss: 1.8015837 accuracy: [16845, 25600] (65 %)
[12,   200] loss: 1.7864041 accuracy: [17223, 25600] (67 %)
[13,   200] loss: 1.7676317 accuracy: [17729, 25600] (69 %)
[14,   200] loss: 1.7489127 accuracy: [18194, 25600] (71 %)
[15,   200] loss: 1.7335578 accuracy: [18592, 25600] (72 %)
[16,   200] loss: 1.7259925 accuracy: [18795, 25600] (73 %)
[17,   200] loss: 1.7134028 accuracy: [19120, 25600]

In [0]:
model.eval()
running_loss = 0.0
correct, total = 0, 0
#correct_epoch, total_epoch = 0, 0
for i, data in enumerate(testloader):
    inp,lab = data[0].to(device),data[1].to(device)
    out = model(inp) 
    c = (torch.argmax(out,1)==lab)
    correct+=torch.sum(c)
    total += 128
    
print('accuracy: ',100*correct/total,''%)    